In [3]:
import nilearn
from nibabel.testing import data_path
from nilearn import datasets, plotting, regions
import brainspace
import pandas as pd
import numpy as np
import os
from nilearn import surface
import nibabel as nib
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#import hcp_utils as hcp
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
from surfplot.plotting import Plot
from icc_utils import *
from hyppo.discrim import DiscrimOneSample

In [4]:
subjects = ["100206", "100307", "100408", "100610", "101006"]
gradient_list = []
for subject in subjects:
    gradient = np.load("HCP/Gradients/%s/%s.pca.ses1.s02mm.npy" % (subject, subject))
    gradient_list.append(gradient.T)

gradients_ses1 = np.stack((gradient_list))

gradient_list = []
for subject in subjects:
    gradient = np.load("HCP/Gradients/%s/%s.pca.ses2.s02mm.npy" % (subject, subject))
    gradient_list.append(gradient.T)

gradients_ses2 = np.stack((gradient_list))

margulies_grads = np.load('margulies_grads_32K.npy').T

In [3]:
from brainspace.gradient.alignment import ProcrustesAlignment

alignment = ProcrustesAlignment(verbose=True, n_iter = 10)
allGradsAlignedObject_ses1 = alignment.fit(list(gradients_ses1), margulies_grads) # aligning to margulies 2016
GradsAligned_ses1 = allGradsAlignedObject_ses1.aligned_
allGradsAlignedObject_ses2 = alignment.fit(list(gradients_ses2), margulies_grads) # aligning to margulies 2016
GradsAligned_ses2 = allGradsAlignedObject_ses2.aligned_

np.save(arr = GradsAligned_ses1, file ="Grads32kAligned2Margulies2016_ses1.npy")
np.save(arr = GradsAligned_ses2, file ="Grads32kAligned2Margulies2016_ses2.npy")

Iteration   0: 1430372.139473
Iteration   1: 0.002696
Iteration   2: 0.000227
Iteration   3: 0.000022
Iteration   4: 0.000002
Iteration   5: 0.000000
Iteration   0: 1430444.230893
Iteration   1: 0.001681
Iteration   2: 0.000064
Iteration   3: 0.000003
Iteration   4: 0.000000


In [5]:
aligned_grads_32k_ses1 = np.load("Grads32kAligned2Margulies2016_ses1.npy")
aligned_grads_32k_ses2 = np.load("Grads32kAligned2Margulies2016_ses2.npy")

In [7]:
disp_ses1 = []
for i in range(len(aligned_grads_32k_ses1)):
    disp_ses_1 = gradDispersion(aligned_grads_32k_ses1[i], 1600, save = True, filename = "disp_1600neighbours_ses1.npy")
    disp_ses1.append(disp_ses_1)

disp_ses2 = []
for i in range(len(aligned_grads_32k_ses2)):
    disp_ses_2 = gradDispersion(aligned_grads_32k_ses2[i], 1600, save = True, filename = "disp_1600neighbours_ses2.npy")
    disp_ses2.append(disp_ses_2)

In [8]:
np.save(arr = np.asarray(disp_ses1), file = "dispersion_ses1.npy")
np.save(arr = np.asarray(disp_ses2), file = "dispersion_ses2.npy")

In [10]:
disp_ses1 = np.load("dispersion_ses1.npy")
disp_ses2 = np.load("dispersion_ses2.npy")

In [34]:

disp_array = np.asarray([np.stack((i, j)) for i, j in zip(disp_ses1, disp_ses2)])
disp_array = np.concatenate((disp_array, disp_array, disp_array))


In [43]:
disp_array.shape
disp_array.mean(axis = 2).shape

(15, 2, 59412)

(15, 2)

In [110]:
x = np.concatenate([np.zeros((50, 2)), np.ones((50, 2))], axis=0)
y = np.concatenate([np.zeros(50), np.ones(50)], axis=0)
#'%.1f, %.2f' % DiscrimOneSample().test(, y) 
#'1.0, 0.00'

In [111]:
x.shape
x = np.concatenate((x1, x2))
x.shape

(100, 2)

(118824, 2)

In [44]:
DiscrimOneSample().test(disp_array.mean(axis = 2), np.asarray(subjects *3)) 

DiscrimOneSampleTestOutput(stat=1.0, pvalue=0.000999000999000999)

In [89]:
label_subj_1 = ["100206"] * 59412
label_subj_2 = ["100201"] * 59412
label_subj = np.asarray(label_subj_1 + label_subj_2)
label_vertex = np.arange(59412).astype(str)
label_vertex = np.concatenate((label_vertex, label_vertex))


In [88]:
label_vertex.shape

(118824,)

In [74]:
disp = np.stack((disp_ses1, disp_ses2))

In [97]:
x1 = disp[:, 0, :].T
x2 = disp[:, 1, :].T

x = np.concatenate((x1, x2))
y = np.stack((label_subj, label_vertex)).T

In [104]:
x.reshape(1, -1).shape

(1, 237648)

In [105]:
DiscrimOneSample().test(x,y) 

ValueError: Expected 2D array, got 1D array instead:
array=[0.0007429  0.00118452 0.00161574 ... 0.00088336 0.0006237  0.00100984].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [15]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])